# Some setup stuff

Here we will be setting up some stuff that will be helpful going forward, such as the location of Bath and the time of the Vernal Equinox of the year 2022.

In [1]:
bath_latitude = 51.37
bath_longitude = -2.36

## Increasing the RA

The Right Ascension in most databases is measured at the vernal equinox, the 20th of March. However, since the Earth moves around the Sun it is not the same all year around. Each day the Earth covers 360/365 degrees of its rotation, and each hour, since it rotates a full 360º it covers 15º. As such, each day the RA increases

$$\Delta RA = \frac{360}{365*15}$$

This cell will end up not being relevant to the calculations, but it is still useful to understand how RA works. This [video](https://www.youtube.com/watch?v=kejolGai7X8&list=PLX2gX-ftPVXU3BfWLJfmXfGD08cIit5CI&index=9) I have found explains it quite well.

## The sidereal time

For these observations we use sidereal time. A sidereal day takes a little less than a solar day, 23h 56 min instead of the usual 24h. This is the time it takes for the Earth to fully rotate around its axis. This is useful because any object crosses the upper meridian when the sidereal time is equal to the object's Right Ascension ([source](https://lco.global/spacebook/sky/sidereal-time/)).

A solar day lasts 24h because it takes that amount of time for the same spot in the Earth to be facing the Sun, since not only does the Earth rotate, but it also moves around the Sun.

Here we are simply computing the sidereal time at Bath right now (whenever _now_ is). This is UTC time, which matches the GMT timezone. At the time of writing this (19th of October), the timezone in Bath is GMT + 1, but that will change on the 30th of October when Daylight saving time is implemented, and we will be back in the GMT/UTC timezone.

In [2]:
# Obtain sidereal time
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy import units as u
import datetime

observing_location = EarthLocation(lat=bath_latitude*u.deg, lon=bath_longitude*u.deg)
observing_time = Time(datetime.datetime.utcnow(), scale='utc', location=observing_location)
LST = observing_time.sidereal_time('mean')

## The sidereal time at the time of observation

Following the same procedure as above, we can compute the sidereal time of the day of observation. I have coded the program to automatically use the day of whenever it's ran on, but it can be changed to any random date. Since we are currently UTC + 1 I have coded the observation to start at 19:15 (20:15 Bath time) and finishing at 1:00 (2:00 Bath time). This has to be changed manually in the 30th, and an hour added to both cases.

Once we have the sidereal times of both the start and the end, we can use it to find which stars are visible.

In [3]:
date_today = datetime.datetime.today().strftime('%Y-%m-%d')
date_tomorrow = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
times = [f'{date_today}T18:00:00', f'{date_tomorrow}T2:00:00']# In UTC, change on 30th October

t = Time(times, format='isot', scale='utc')

In [4]:
import re

start_end_times = re.split(' ',str(t.sidereal_time('apparent', bath_longitude)))
start_sidereal_time = start_end_times[0].strip('[]')[:-5] # Keeping it to 4 decimals
end_sidereal_time = start_end_times[1].strip('[]')[:-5]

# Formatting it to hh mm ss so it has the same format as the RA

start_sidereal_time = start_sidereal_time.replace('h', ' ').replace('m', ' ')
end_sidereal_time = end_sidereal_time.replace('h', ' ').replace('m', ' ')

## Getting the right ascension of all stars of our query

Here we are importing the dataframe with the stars of the Monson et al paper, which I saved as a pickle in that notebook. We also split the Right Ascension column into hours minutes and seconds to make calculations later on a bit simpler. With such a small dataset, the impact on performance is not huge, and it makes things much easier.

In [5]:
import pandas as pd
simbad_df = pd.read_pickle('Monson_stars_simbad.pkl')

## Getting the declination of all stars of our query

Same thing as before, but now we do it with the declination column. It's split into three: degrees, minutes and seconds.

In [6]:
simbad_df['DEC degrees'] = simbad_df['DEC'].str[0:3].astype(float)
simbad_df['DEC minutes'] = simbad_df['DEC'].str[4:6].astype(float)
simbad_df['DEC seconds'] = simbad_df['DEC'].str[7:].astype(float)

# Finding the hour angle of different stars

In order to actually compute which stars are visible we are using the hour angle, i.e. how far away from the meridian it is at the beginning and end of our observations. In order to calculate the hour angle we are using the following formula:

$$\mathrm{LHA}_{\mathrm{object}} = \mathrm{LST}-\alpha_{\mathrm{object}}$$

where $\mathrm{LHA}_{\mathrm{object}}$ is the Local Hour Angle of the given object, LST is the local sidereal time and $\alpha_{\mathrm{object}}$ is the Right Ascension of the object. We split it into hours, minutes and seconds and then put it back together.

In [7]:
def formatTimedelta(td):
    """
    format a timedelta object to string, in HH:MM:SS format (seconds floored).
    negative timedeltas will be prefixed with a minus, '-'.
    """
    total = td.total_seconds()
    prefix, total = ('-', total*-1) if total < 0 else ('', total)
    h, r = divmod(total, 3600)
    m, s = divmod(r, 60)
    return f"{prefix}{int(h):02d}:{int(m):02d}:{int(s):02d}"

simbad_df['start sidereal time'] = start_sidereal_time
simbad_df['end sidereal time'] = end_sidereal_time

FMT = '%H %M %S.%f'
simbad_df['RA'] = pd.to_datetime(simbad_df['RA'], format = FMT)
simbad_df['start sidereal time'] = pd.to_datetime(simbad_df['start sidereal time'], format = FMT)
simbad_df['end sidereal time'] = pd.to_datetime(simbad_df['end sidereal time'], format = FMT)




simbad_df['Start hour angle'] = ((simbad_df['start sidereal time'] - simbad_df['RA'])).apply(formatTimedelta)
simbad_df['End hour angle'] = ((simbad_df['end sidereal time']-simbad_df['RA'])).apply(formatTimedelta)
# simbad_df['Time observable'] = (simbad_df['Start hour angle'] - simbad_df['End hour angle'])

In [8]:


fmt = '%H:%M:%S'
neg_fmt = '-%H:%M:%S'

midday = datetime.datetime.strptime('1 Jan 1900 12:00:00', '%d %b %Y %H:%M:%S')
midnight = datetime.datetime.strptime('2 Jan 1900 00:00:00', '%d %b %Y %H:%M:%S')

start_ha_series = simbad_df['Start hour angle'].reset_index(drop = True)
end_ha_series = simbad_df['End hour angle'].reset_index(drop = True)
for start_ha, start_ha_index, end_ha, end_ha_index in zip(start_ha_series, start_ha_series.index, end_ha_series, end_ha_series.index):
    try:
        start_dt = datetime.datetime.strptime(start_ha, fmt)
        if start_dt > midday:
            corrected_start_time = midnight-start_dt
            corrected_start_time_string = f"-{corrected_start_time}"
        else:
            corrected_start_time_string = start_dt.strftime('%H:%M:%S')


    except ValueError:
        start_dt = datetime.datetime.strptime(start_ha, neg_fmt)
        if start_dt < midday:
            corrected_start_time_string = f"-{start_dt.strftime('%H:%M:%S')}"
        else:
            corrected_start_time = midnight-start_dt
            corrected_start_time_string = f"{corrected_start_time}"
    finally:
        start_ha_series.iloc[start_ha_index] = corrected_start_time_string

    try:
        end_dt = datetime.datetime.strptime(end_ha, fmt)
        if end_dt > midday:
            corrected_end_time = midnight-end_dt
            corrected_end_time_string = f"-{corrected_end_time}"
        else:
            corrected_end_time_string = end_dt.strftime('%H:%M:%S')

    except ValueError:
        end_dt = datetime.datetime.strptime(end_ha, neg_fmt)
        if end_dt < midday:
            corrected_end_time_string = f"-{end_dt.strftime('%H:%M:%S')}"
        else:
            corrected_end_time = midnight-end_dt
            corrected_end_time_string = f"{corrected_end_time}"
    finally:
        end_ha_series.iloc[end_ha_index] = corrected_end_time_string

In [9]:
simbad_df['Start hour angle folded'] = start_ha_series.values
simbad_df['End hour angle folded'] = end_ha_series.values

In [23]:
simbad_df

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,SCRIPT_NUMBER_ID,DEC degrees,DEC minutes,DEC seconds,start sidereal time,end sidereal time,Start hour angle,End hour angle,Start hour angle folded,End hour angle folded
0,V* SW And,1900-01-01 00:23:43.089600,+29 24 03.626,14,14,0.0216,0.0176,90,A,O,...,1,29.0,24.0,3.626,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,00:31:03,08:32:22,00:31:03,08:32:22
0,V* XX And,1900-01-01 01:17:27.414200,+38 57 02.047,14,14,0.0149,0.0152,90,A,O,...,1,38.0,57.0,2.047,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-00:22:40,07:38:38,-00:22:40,07:38:38
0,V* WY Ant,1900-01-01 10:16:04.946100,-29 43 42.417,14,14,0.0162,0.0159,90,A,O,...,1,-29.0,43.0,42.417,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-09:21:17,-01:19:59,-09:21:17,-01:19:59
0,V* X Ari,1900-01-01 03:08:30.884600,+10 26 45.218,14,14,0.0171,0.0145,90,A,O,...,1,10.0,26.0,45.218,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-02:13:43,05:47:35,-02:13:43,05:47:35
0,V* ST Boo,1900-01-01 15:30:39.229900,+35 47 04.311,14,14,0.0154,0.0173,90,A,O,...,1,35.0,47.0,4.311,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-14:35:52,-06:34:33,9:24:08,-06:34:33
0,V* UY Boo,1900-01-01 13:58:46.336700,+12 57 06.457,14,14,0.0193,0.0152,90,A,O,...,1,12.0,57.0,6.457,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-13:03:59,-05:02:40,10:56:01,-05:02:40
0,V* RR Cet,1900-01-01 01:32:08.173100,+01 20 30.230,14,14,0.0152,0.0118,90,A,O,...,1,1.0,20.0,30.230,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-00:37:21,07:23:57,-00:37:21,07:23:57
0,V* W Crt,1900-01-01 11:26:29.645200,-17 54 51.700,14,14,0.0198,0.0176,90,A,O,...,1,-17.0,54.0,51.700,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-10:31:42,-02:30:23,-10:31:42,-02:30:23
0,V* UY Cyg,1900-01-01 20:56:28.303000,+30 25 40.327,14,14,0.0102,0.0154,90,A,O,...,1,30.0,25.0,40.327,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-20:01:41,-12:00:22,3:58:19,11:59:38
0,V* XZ Cyg,1900-01-01 19:32:29.305000,+56 23 17.494,14,14,0.0135,0.0147,90,A,O,...,1,56.0,23.0,17.494,1900-01-01 00:54:47.081900,1900-01-01 08:56:05.936400,-18:37:42,-10:36:23,5:22:18,-10:36:23


# Finding the observable stars



## Circumpolar star

A circumpolar star never sets, but it rotates around the celestial north pole. The amount of circumpolar stars depends on latitude. There are, for example, no circumpolar stars in the equator, and all of the stars are circumpolar at the poles. A star will be circumpolar if
$$ \theta + \delta > 90º$$

where $\theta$ is the latitude and $\delta$ is the declination.

In [11]:
circumpolar_stars = simbad_df[simbad_df['DEC degrees'] + bath_latitude > 90]

## Discriminating based on RA and Dec

Aside from the circumpolar stars, we want something that will be close to the Meridian at the time of observation. This means an hour angle of $\pm3$ hrs or less, which amounts to $\pm 45$º in the East to West plane. Each hour it will move 15º, so that something that starts with an HA = 3hrs at 8pm will cross the meridian at 11pm. Using the hour angle allows us to circumvent calculating which stars are not observable in this coordinate, since anything not visible will have an hour angle too large.

In terms of declination, we want something that is visible from Bath. Since we are not at the equator, the stars will not be where its declination says (in the North to South direction), but instead the following correction must be applied:

$$\mathrm{Dec}_{\mathrm{Bath}} = \mathrm{Dec} - \mathrm{lat}$$

This will tell us what angle the star has with respect to the zenith. To avoid light pollution and trees we can limit the observable declination to 30º in each direction, so that we are left with a small circle in the sky with our observable stars.

In [12]:
def compare_ha(hour_angle):
    h, m, s = re.split(':',hour_angle)
    return 3 > int(h) > -3

def visible_for_calculator(hour_angle):
    h, m, s = re.split(':', hour_angle)
    h = int(h)
    m = int(m)
    s = int(s)
    if 3 > h > 0:
        seconds = 60 - s
        minutes = 60 - m - 1
        hours = 3 - h - 1
        return(f'{hours}:{minutes}:{seconds}')
    elif -3 < h < 0:
        seconds = s
        minutes = m
        hours = 3 - h
        return(f'{hours}:{minutes}:{seconds}')
    elif h == 0:
        if hour_angle[0] == '-':
            seconds = s
            minutes = m
            hours = 3
            return(f'{hours}:{minutes}:{seconds}')
        else:
            seconds = 60 - s
            minutes = 60 - m - 1
            hours = 2
            return(f'{hours}:{minutes}:{seconds}')
    elif -3 < h + 6 <= 3:
        final_hour = h + 6
        foo = final_hour + 3 - 1
        minutes = 60 - m - 1
        seconds = 60 - s
        return(f'{foo}:{minutes}:{seconds}')
    else:
        return("Not observable")



In [24]:
pd.options.mode.chained_assignment = None

ra_observable = simbad_df[simbad_df['Start hour angle folded'].apply(compare_ha)]
ra_observable['Visible for'] = ra_observable['Start hour angle folded'].apply(visible_for_calculator)
print(ra_observable.shape)

dec_observable = ra_observable[(ra_observable['DEC degrees']-bath_latitude < 30) & (ra_observable['DEC degrees']-bath_latitude > -30)]

print(dec_observable.shape)

# dec_observable = pd.concat([dec_observable, circumpolar_stars])
# Here I just get rid of the columns I made for the operations
dec_observable = dec_observable.drop(columns = ['DEC degrees', 'DEC minutes', 'DEC seconds', 'start sidereal time', 'end sidereal time', 'Start hour angle', 'End hour angle'])


(13, 33)
(4, 33)


In [14]:
# Drop the duplicates in case there are any
dec_observable = dec_observable.drop_duplicates()

In [15]:
def organizer(string):
    h, m, s = re.split(':', string)
    h = int(h)
    m = int(m)
    s = int(s)
    ts = h * 3600 + m * 60 + s
    return ts

dec_observable['Visible for in seconds'] = dec_observable['Visible for'].apply(organizer)
dec_observable.sort_values(by=['Visible for in seconds'], ascending = False)

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,Distance_bibcode,FLUX_K,FLUX_G,FLUX_R,FLUX_B,SCRIPT_NUMBER_ID,Start hour angle folded,End hour angle folded,Visible for,Visible for in seconds
0,V* XX And,1900-01-01 01:17:27.414200,+38 57 02.047,14,14,0.0149,0.0152,90,A,O,...,2020yCat.1350....0G,9.336,10.681514,NaN,11.00,1,-00:22:40,07:38:38,3:22:40,12160
0,V* RU Psc,1900-01-01 01:14:26.037100,+24 24 56.367,14,14,0.0249,0.0164,90,A,O,...,2020yCat.1350....0G,9.068,10.129585,NaN,10.34,1,-00:19:38,07:41:39,3:19:38,11978
0,V* SW And,1900-01-01 00:23:43.089600,+29 24 03.626,14,14,0.0216,0.0176,90,A,O,...,2020yCat.1350....0G,8.444,9.705101,NaN,9.34,1,00:31:03,08:32:22,2:28:57,8937
0,V* RZ Cep,1900-01-01 22:39:13.176800,+64 51 30.611,14,14,0.0108,0.0099,90,A,O,...,2020yCat.1350....0G,7.968,9.294270,9.3,9.48,1,2:15:34,10:16:53,0:44:26,2666


In [16]:
# To do:
# Fix observable for calculation so that it accounts for start and end times

In [17]:
16 * 10**((10.681514 - 10.7)/2.5)

15.729886271849844

In [18]:
for id, flux in set(zip(dec_observable['MAIN_ID'], dec_observable['FLUX_G'])):
    print(f'{id}:{16 * 10**((flux - 10.858340)/2.5):.2f} seconds exposure')

V* SW And:5.53 seconds exposure
V* RU Psc:8.18 seconds exposure
V* RZ Cep:3.79 seconds exposure
V* XX And:13.60 seconds exposure


In [19]:
dec_observable.to_csv('Observable_tonight.csv')

In [20]:
from astropy.io import fits

path = 'C:/Users/cuco2/OneDrive/Escritorio/Physics/Year 4/Final year project/MphysProject/2022_03_20/RV_UMa/Light/Luminance/RV_Uma_Light_Luminance_16_secs_011.fits'
test = fits.open(path)
test.info()
hdr =test[0].header
hdr

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/cuco2/OneDrive/Escritorio/Physics/Year 4/Final year project/MphysProject/2022_03_20/RV_UMa/Light/Luminance/RV_Uma_Light_Luminance_16_secs_011.fits'